In [1]:
!pip install underthesea==6.8.4
import pandas as pd
import re
from underthesea import word_tokenize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.8 MB/s eta 0:00:00


In [2]:
df = pd.read_csv("/kaggle/input/cogdis-data/Combined_Data.csv")

In [3]:
neg_patterns = [
    "không thể", "chẳng thể", "chẳng bao giờ", "chưa bao giờ",
    "không bao giờ", "không còn", "chẳng còn", "không làm được",
    "chẳng làm được", "không thể chịu nổi", "không dám", "không đủ",
    "chẳng hề", "chưa từng", "chẳng từng", "chẳng muốn", "không muốn"
]

def detect_neg_pattern(text):
    text = text.lower()
    for pat in neg_patterns:
        if re.search(r"\btôi\b.{0,10}" + pat, text):  # khoảng cách tối đa 10 token
            return 1
    return 0

causal_markers = [
    "vì", "bởi vì", "do", "do đó", "do vậy", "do thế", "nên", "cho nên",
    "vì thế", "vì vậy", "khiến", "thành ra", "dẫn đến", "hậu quả là",
    "do nguyên nhân", "do việc", "bởi thế", "tại vì"
]

def detect_causal_structure(text):
    text = text.lower()
    for cm in causal_markers:
        if cm in text:
            return 1
    return 0

neg_words = [
    "không", "chẳng", "chả", "chưa", "chẳng hề", "đâu", "không hề", "không bao giờ",
    "chẳng bao giờ", "không thể", "không còn", "chưa từng", "không dám", "chẳng thể"
]

def negation_ratio(text):
    tokens = word_tokenize(text.lower())
    if len(tokens) == 0:
        return 0
    neg_count = sum(1 for t in tokens if t in neg_words)
    return round(neg_count / len(tokens), 3)

pronouns = [
    "tôi", "mình", "ta", "chúng tôi", "chúng ta", "bản thân", 
    "anh ấy", "cô ấy", "họ", "bạn", "chúng nó", "tụi nó", "chúng mày"
]

def pronoun_count(text):
    tokens = word_tokenize(text.lower())
    return sum(1 for t in tokens if t in pronouns)

df["has_neg_pattern"] = df["Original Text"].apply(detect_neg_pattern)
df["has_causal_structure"] = df["Original Text"].apply(detect_causal_structure)
df["negation_ratio"] = df["Original Text"].apply(negation_ratio)
df["pronoun_count"] = df["Original Text"].apply(pronoun_count)

syntactic_features = df[[
    "Original Text", 
    "has_neg_pattern", 
    "has_causal_structure", 
    "negation_ratio", 
    "pronoun_count"
]]

syntactic_features.head(10)

,Original Text,has_neg_pattern,has_causal_structure,negation_ratio,pronoun_count
0,"Cô ấy luôn có những hành động kỳ lạ, ví dụ như...",0,1,0.051,4
1,"Nếu bố mẹ tôi biết tôi đang vật lộn thế nào, h...",0,0,0.033,5
2,"Trong năm thứ 4 và thứ 5 của sự nghiệp, tôi kh...",1,1,0.052,7
3,"Tôi muốn bị ốm, và tôi biết điều đó thật khủng...",1,0,0.053,4
4,Trước khi chúng tôi bắt đầu hẹn hò và trong nă...,0,0,0.000,2
5,"Vì không có gì tôi làm là đủ tốt, chúng tôi đã...",0,1,0.042,3
6,"Hàng ngày và đôi khi trước khi ngủ, tôi luôn t...",0,0,0.000,1
7,"Gần đây, nó đã biến thành OCD với các ý nghĩ x...",0,0,0.020,3
8,"Tôi không biết bắt đầu từ đâu hay nói gì, nhưn...",0,0,0.053,5
9,"Tôi chưa nói với ai điều này, tôi biết họ sẽ n...",0,0,0.042,3


In [4]:
syntactic_features.to_csv("syntactic_features.csv", index=False)